<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/homework/04-advanced-ML/HW/hw3/vw_vs_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vowpal Wabbit

*Поработаем с той же задачей и с теми же данными, на которых мы настраивали параметры xgboost.*

*Посмотрим, справятся ли линейные алгоритмы, реализованные в vowpal wabbit, с этой задачей.*

## 1.

In [1]:
!apt-get install vowpal-wabbit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libvw0
Suggested packages:
  vowpal-wabbit-doc
The following NEW packages will be installed:
  libvw0 vowpal-wabbit
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 797 kB of archives.
After this operation, 3,034 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libvw0 amd64 8.5.0.dfsg1-1 [748 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 vowpal-wabbit amd64 8.5.0.dfsg1-1 [49.1 kB]
Fetched 797 kB in 6s (127 kB/s)
Selecting previously unselected package libvw0.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../libvw0_8.5.0.dfsg1-1_amd64.deb ...
Unpacking libvw0 (8.5.0.dfsg1-1) ...
Selecting previously unselected package vowpal-wabbit.
Preparing to unpack .../vowpal-wabbit_8.5.0.dfsg1-1_amd64.deb

In [2]:
!wget https://www.dropbox.com/s/eg9wt98ak9n91w2/train_medium.csv?dl=0
!wget https://www.dropbox.com/s/mji0jvu5wo34p2w/test_medium.csv?dl=0

--2019-01-29 20:23:11--  https://www.dropbox.com/s/eg9wt98ak9n91w2/train_medium.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/eg9wt98ak9n91w2/train_medium.csv [following]
--2019-01-29 20:23:11--  https://www.dropbox.com/s/raw/eg9wt98ak9n91w2/train_medium.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc58247bfa67f6feaa7cd2315cb8.dl.dropboxusercontent.com/cd/0/inline/AaVtM1bpq_OnL25KelC9_DUe6sG-L8NvyjO8WVVicJrhOPo3Z_36uoqfDsOdq71NdWamRB5eTsx4VXQQaDJJjSWdOIgkChwPMgkYpC-T6ChFeg/file# [following]
--2019-01-29 20:23:12--  https://uc58247bfa67f6feaa7cd2315cb8.dl.dropboxusercontent.com/cd/0/inline/AaVtM1bpq_OnL25KelC9_DUe6sG-L8NvyjO8WVVicJrhOPo3Z_36uoqfDsOdq71NdWamRB5eTsx4VXQQaDJJjSWdOIgkChwPMgkYpC-T6ChFeg/file
Re

In [3]:
!ls

 sample_data  'test_medium.csv?dl=0'  'train_medium.csv?dl=0'


In [0]:
import numpy as np
import pandas as pd
import regex as re

In [5]:
X_train = pd.read_csv('train_medium.csv?dl=0')
X_test = pd.read_csv('test_medium.csv?dl=0')

X_train.shape, X_test.shape

((65265, 51), (21755, 51))

In [6]:
X_train.head()

,Unnamed: 0,Existing_EMI,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,...,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2,Disbursed
0,7881,0.0,100000.0,3.0,29000,4,11,29,0,0,...,0,0,0,0,0,1,0,1,0,1.0
1,189,21000.0,450000.0,5.0,48000,1,0,27,1,1,...,0,0,0,0,1,0,0,1,0,0.0
2,64000,3832.0,100000.0,3.0,25000,5,11,44,0,0,...,0,0,0,1,0,1,1,0,0,0.0
3,61629,0.0,200000.0,3.0,22500,3,1,26,1,1,...,0,0,0,1,0,1,1,0,0,0.0
4,23212,0.0,50000.0,1.0,17943,1,0,25,1,1,...,0,0,0,0,1,0,0,1,0,0.0


In [7]:
X_test['Disbursed'].value_counts()

0.0    21431
1.0      324
Name: Disbursed, dtype: int64

*Измените функцию save_to_vw из предыдущего ноутбука (см.ниже) так, чтобы она записывала файлы train_medium и test_medium в нужном формате. Не забудьте, что столбец 'Disbursed' является целевым, поэтому а) он определяет значение переменной target, b) его не нужно записывать в файл-матрицу с признаками.*

In [0]:
def fomrat_vw(row):
    target = row['Disbursed']
    result ="{1} '{0:.0f}|".format(target, 1 if 0 < target else -1)
    result +=' '.join([ ' {}:{}'.format(re.sub('\W+','_',name), value) for name, value in row.drop('Disbursed').iteritems() ])
    return result

In [9]:
%%time

vw_train = X_train.apply(fomrat_vw, axis=1)
vw_test = X_test.apply(fomrat_vw, axis=1)

CPU times: user 55.7 s, sys: 97.6 ms, total: 55.8 s
Wall time: 55.8 s


In [10]:
vw_train.head()

0    1 '1| Unnamed_0:7881.0  Existing_EMI:0.0  Loan...
1    -1 '0| Unnamed_0:189.0  Existing_EMI:21000.0  ...
2    -1 '0| Unnamed_0:64000.0  Existing_EMI:3832.0 ...
3    -1 '0| Unnamed_0:61629.0  Existing_EMI:0.0  Lo...
4    -1 '0| Unnamed_0:23212.0  Existing_EMI:0.0  Lo...
dtype: object

In [0]:
np.savetxt('train_medium.vw', vw_train.values, '%s')
np.savetxt('test_medium.vw', vw_test.values, '%s')

In [12]:
!ls

 sample_data		 test_medium.vw		  train_medium.vw
'test_medium.csv?dl=0'	'train_medium.csv?dl=0'


*Скопируйте полученные файлы в папку share и обучите vowpal wabbit. Например, предсказания классов можно сохранять в файл с именем preds.txt, а предсказания вероятностей - в файл probs.txt.*

*Поэкспериментируйте с параметрами vowpal wabbit и постарайтесь добиться наилучшего качества на тесте.*

*Постарайтесь, насколько это возможно, приблизить ROC-AUC к значению, полученному после оптимизации xgboost.*

In [13]:
!head -n 3 train_medium.vw

1 '1| Unnamed_0:7881.0  Existing_EMI:0.0  Loan_Amount_Applied:100000.0  Loan_Tenure_Applied:3.0  Monthly_Income:29000.0  Var4:4.0  Var5:11.0  Age:29.0  EMI_Loan_Submitted_Missing:0.0  Interest_Rate_Missing:0.0  Loan_Amount_Submitted_Missing:0.0  Loan_Tenure_Submitted_Missing:0.0  Processing_Fee_Missing:0.0  Device_Type_0:0.0  Device_Type_1:1.0  Filled_Form_0:1.0  Filled_Form_1:0.0  Gender_0:0.0  Gender_1:1.0  Var1_0:0.0  Var1_1:0.0  Var1_2:0.0  Var1_3:0.0  Var1_4:0.0  Var1_5:0.0  Var1_6:0.0  Var1_7:0.0  Var1_8:0.0  Var1_9:0.0  Var1_10:0.0  Var1_11:1.0  Var1_12:0.0  Var1_13:0.0  Var1_14:0.0  Var1_15:0.0  Var1_16:0.0  Var1_17:0.0  Var1_18:0.0  Var2_0:0.0  Var2_1:1.0  Var2_2:0.0  Var2_3:0.0  Var2_4:0.0  Var2_5:0.0  Var2_6:0.0  Mobile_Verified_0:0.0  Mobile_Verified_1:1.0  Source_0:0.0  Source_1:1.0  Source_2:0.0
-1 '0| Unnamed_0:189.0  Existing_EMI:21000.0  Loan_Amount_Applied:450000.0  Loan_Tenure_Applied:5.0  Monthly_Income:48000.0  Var4:1.0  Var5:0.0  Age:27.0  EMI_Loan_Submitted_Missi

## 2

### 2.1

Сначала попробуем с одинаковыми весами для классов и параметрами по умолчанию.

In [14]:
!vw -d train_medium.vw -c -k -f model.vw --passes 10 --link logistic

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train_medium.vw.cache
Reading datafile = train_medium.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000       15
2.120413 3.240826            2            2.0  -1.0000   0.6900       20
1.706689 1.292964            4            4.0  -1.0000   0.3142       18
0.864536 0.022383            8            8.0  -1.0000   0.2769       19
0.757420 0.650304           16           16.0  -1.0000   0.3284       16
0.380981 0.004542           32           32.0  -1.0000   0.2689       15
0.194584 0.008186           64           64.0  -1.0000   0.2689       20
0.097948 0.001312          128          128.0  -1.0000   0.2689       19
0.049074 0.000201          256          256.0  -1.0000 

In [15]:
!ls

 model.vw		 test_medium.vw		  train_medium.vw.cache
 sample_data		'train_medium.csv?dl=0'
'test_medium.csv?dl=0'	 train_medium.vw


In [16]:
!vw -d test_medium.vw -i model.vw -t -p probs.txt

only testing
predictions = probs.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test_medium.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.002402 0.002402            1            1.0  -1.0000   0.2787       18
0.001201 0.000001            2            2.0  -1.0000   0.2691       18
0.002775 0.004349            4            4.0  -1.0000   0.2877       16
0.004098 0.005420            8            8.0  -1.0000   0.2835       20
0.004410 0.004722           16           16.0  -1.0000   0.2926       16
0.004201 0.003992           32           32.0  -1.0000   0.2775       20
0.061743 0.119286           64           64.0   1.0000   0.2883       16
0.120200 0.178657          128          128.0  -1.0000   0.2858       18
0.076308 0.032417          256          256.0  -1.0000   0.2689       13
0.075953 0.075598         

In [17]:
!head -5 probs.txt

0.278686 0
0.269102 0
0.268941 0
0.287668 0
0.279070 0


In [0]:
from sklearn import metrics

def print_scores(dataset_name, y_true, y_pred, y_proba):
    print("[{}] {}".format(dataset_name, dict(zip(*np.unique(y_pred, return_counts=True)))))
    print("Accuracy : {:.3f}".format(metrics.accuracy_score(y_true, y_pred)))
    print("Balanced accuracy : {:.3f}".format(metrics.balanced_accuracy_score(y_true, y_pred)))
    print("AUC Score [class {:.0f}]: {:.3f}".format(1, metrics.roc_auc_score(y_true, y_proba)))
    print(metrics.classification_report(y_true, y_pred, digits=3))
    print(60 * "*")
    
def get_preds(y_proba, proba_treshold = 0.5):
     return np.array([0, 1])[(y_proba > proba_treshold).values.astype('int')]    

In [19]:
probs = pd.read_csv('probs.txt', header=None, names=['Proba', 'Label'], sep=' ')

probs[probs['Label'] == 1].head()

,Proba,Label
63,0.288329,1
73,0.287613,1
87,0.291979,1
88,0.274852,1
220,0.290738,1


In [21]:
np.unique(get_preds(probs['Proba']), return_counts=True)

(array([0]), array([21755]))

In [22]:
print_scores('vw', X_test['Disbursed'], get_preds(probs['Proba']), probs['Proba'])

[vw] {0: 21755}
Accuracy : 0.985
Balanced accuracy : 0.500
AUC Score [class 1]: 0.729
              precision    recall  f1-score   support

         0.0      0.985     1.000     0.992     21431
         1.0      0.000     0.000     0.000       324

   micro avg      0.985     0.985     0.985     21755
   macro avg      0.493     0.500     0.496     21755
weighted avg      0.970     0.985     0.978     21755

************************************************************


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
def print_metrics(vw_probs, proba_treshold = 0.5):
    probs = pd.read_csv(vw_probs, header=None, names=['Proba', 'Label'], sep=' ')
    y_pred = get_preds(probs['Proba'], proba_treshold)
    print_scores('vw', X_test['Disbursed'], y_pred, probs['Proba'])

### 2.2
Увеличим вес класса **1**. Это позволяет увеличить **recall** по редкому классу.

In [24]:
!vw -d train_medium.vw -c -k -f model.vw --passes 10 --link logistic --classweight 1:16 --quiet
!vw -d test_medium.vw -i model.vw -t -p probs.txt --quiet

print_metrics('probs.txt')

[vw] {0: 20937, 1: 818}
Accuracy : 0.954
Balanced accuracy : 0.597
AUC Score [class 1]: 0.771
              precision    recall  f1-score   support

         0.0      0.988     0.965     0.977     21431
         1.0      0.090     0.228     0.130       324

   micro avg      0.954     0.954     0.954     21755
   macro avg      0.539     0.597     0.553     21755
weighted avg      0.975     0.954     0.964     21755

************************************************************


### 2.3
Дальнейшее увеличение веса класса **1** отрицательно сказывается на других метриках, в первую очередь на общем **accuracy** - слишком много false alarm-ов.

In [25]:
!vw -d train_medium.vw -c -k -f model.vw --passes 10 --link logistic --classweight 1:50 --quiet
!vw -d test_medium.vw -i model.vw -t -p probs.txt --quiet

print_metrics('probs.txt')

[vw] {0: 14497, 1: 7258}
Accuracy : 0.674
Balanced accuracy : 0.708
AUC Score [class 1]: 0.768
              precision    recall  f1-score   support

         0.0      0.994     0.673     0.802     21431
         1.0      0.033     0.744     0.064       324

   micro avg      0.674     0.674     0.674     21755
   macro avg      0.514     0.708     0.433     21755
weighted avg      0.980     0.674     0.791     21755

************************************************************


### 2.4
Попробуем изменять некоторые другие параметры vowpal wabbit и посмотреть значения метрик.

In [26]:
!vw -d train_medium.vw -c -f model.vw --passes 10 -l 0.1 --link logistic --classweight 1:16 --quiet
!vw -d test_medium.vw -i model.vw -t -p probs.txt --quiet

print_metrics('probs.txt')

[vw] {0: 21244, 1: 511}
Accuracy : 0.966
Balanced accuracy : 0.570
AUC Score [class 1]: 0.789
              precision    recall  f1-score   support

         0.0      0.987     0.979     0.983     21431
         1.0      0.102     0.160     0.125       324

   micro avg      0.966     0.966     0.966     21755
   macro avg      0.544     0.570     0.554     21755
weighted avg      0.974     0.966     0.970     21755

************************************************************


In [27]:
!vw -d train_medium.vw -c -f model.vw --passes 10 -l 0.1 --link logistic --loss_function=logistic --classweight 1:16 --quiet
!vw -d test_medium.vw -i model.vw -t -p probs.txt --quiet

print_metrics('probs.txt')

[vw] {0: 21085, 1: 670}
Accuracy : 0.961
Balanced accuracy : 0.596
AUC Score [class 1]: 0.787
              precision    recall  f1-score   support

         0.0      0.988     0.972     0.980     21431
         1.0      0.106     0.219     0.143       324

   micro avg      0.961     0.961     0.961     21755
   macro avg      0.547     0.596     0.561     21755
weighted avg      0.975     0.961     0.967     21755

************************************************************


In [28]:
!vw -d train_medium.vw -c -f model.vw --passes 10 -l 0.05 --link logistic --loss_function=logistic --classweight 1:20 --quiet
!vw -d test_medium.vw -i model.vw -t -p probs.txt --quiet

print_metrics('probs.txt')

[vw] {0: 20809, 1: 946}
Accuracy : 0.950
Balanced accuracy : 0.624
AUC Score [class 1]: 0.784
              precision    recall  f1-score   support

         0.0      0.989     0.960     0.974     21431
         1.0      0.098     0.287     0.146       324

   micro avg      0.950     0.950     0.950     21755
   macro avg      0.544     0.624     0.560     21755
weighted avg      0.976     0.950     0.962     21755

************************************************************


## 3. Выводы

Значение `ROC_AUC` получается достаточно близкое к `XGBoost`.
Увеличение веса редкого класса **1** позволяет немного (хотя и заметно по сравнению с 0) увеличить **recall** по нему.
Также приятно порадовала скорость обучения.